# Decision Tree Lab

In [618]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import os
import csv

## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits. Otherwise, even with approaches like pruning below, the SSE type of overfit could still hurt us.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [619]:
class DTClassifier(BaseEstimator,ClassifierMixin):
    
    def __init__(self, feature_names=None):
        self.feature_names = feature_names
        
    def fit(self, X, y):
        self.split_gains = np.array([])
        self.tree = self.make_tree(X, y, self.feature_names)
        return self
    
    def make_tree(self, X, y, feature_names):
        totalEntropy = self.entropy(y, 1)

        if len(X) == 0 or len(X[0]) == 0:
            values, counts = np.unique(y, return_counts=True)
            return values[np.argmax(counts)]
        elif (y == y[0]).sum() == len(X):
            if isinstance(y[0], (int, np.integer)) or isinstance(y[0], np.str_):
                return y[0]
            else:
                return y[0][0]
        else:
            info_gain = self.calc_info_gain(X, y)
            gain = np.absolute(info_gain - totalEntropy)
            best_feature = np.argmax(gain)
            self.split_gains = np.append(self.split_gains, gain[best_feature])
            tree = {feature_names[best_feature]:{}}
            values = []
            for row in X:
                if row[best_feature] not in values:
                    values.append(row[best_feature])
            for value in values:
                new_X = []
                new_y = []
                index = 0
                for row in X:
                    if row[best_feature]==value:
                        if best_feature == 0:
                            new_row = row[1:]
                            new_feature_names = feature_names[1:]
                        elif best_feature == len(X[0]):
                            new_row = row[:-1]
                            new_feature_names = feature_names[:-1]
                        else:
                            new_row = row[:best_feature]
                            new_row = np.append(new_row, row[best_feature+1:])
                            new_feature_names = feature_names[:best_feature]
                            new_feature_names = np.append(new_feature_names, feature_names[best_feature+1:])
                        new_X.append(new_row)
                        new_y.append(y[index])
                    index +=1
                subtree = self.make_tree(np.array(new_X), np.array(new_y), new_feature_names)
                
                tree[feature_names[best_feature]][value] = subtree     
        return tree
    
    def calc_info_gain(self, X, y):
        
        total_info = np.array([])
        for column in X.T:
            value,counts = np.unique(column, return_counts=True)
            feature_info = 0
            for indx, i in enumerate(value):
                feature_info += dt.entropy(y[column == i], counts[indx]/X.shape[0])
            total_info = np.append(total_info, feature_info)
        return total_info
        
        
        
    def entropy(self, y, attribute_prob):
        value,counts = np.unique(y, return_counts=True)
        norm_counts = counts / counts.sum()
        return -attribute_prob*(norm_counts * np.log(norm_counts)/np.log(2)).sum()
    
    def print_tree(self, t, s):
        if not isinstance(t,dict) and not isinstance(t,list):
            print("\t"*s+str(t))
        else:
            for key in t:
                print("\t"*s+str(key))
                if not isinstance(t,list):
                    self.print_tree(t[key],s+1)
    
    def predict(self, X):
        prediction = np.array([])
        for row in X:
            prediction = np.append(prediction, self.predict_row(self.tree, row))
        return prediction, prediction.shape[0]
    
    def predict_row(self, tree, row):
        if type(tree) == type("string") or isinstance(tree, (int, np.integer)):
            return tree
        else:
            key = list(tree)[0]
            for i in range(len(self.feature_names)):
                if self.feature_names[i] == key:
                    break
                    
            try:
                t = tree[key][row[i]]
                return self.predict_row(t, row)
            except:
                return None

    def score(self, X, y):
        prediction, length = self.predict(X)
        correct = 0
        #print(prediction)
        #print(y)
        for i in range(length):
            if prediction[i] == y[i]:
                correct += 1
        
        return correct/length
    
def shuffle_data(X, y):
    xy = np.column_stack((X,y))
    size = xy.shape[0]
    for i in range(size):
        swapIndex = np.random.randint(0,size)
        xy[i], xy[swapIndex] = xy[swapIndex], xy[i].copy()
    return xy[:,:-1], xy[:,-1]

## 1.1 Debug

Debug your model by training on the lenses dataset: [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses.arff)

Test your model on the lenses test set: [Debug Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses_test.arff)

Parameters:
(optional) counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv)

Split Information Gains (These do not need to be in this exact order):

[0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits should look like, and the corresponding child node predictions:

Decision Tree:
<pre>
feature_3 = 0:
	feature_2 = 0:
		feature_0 = 0:
			prediction: 2
		feature_0 = 1:
			feature_1 = 0:
				prediction: 2
			feature_1 = 1:
				prediction: 1
		feature_0 = 2:
			prediction: 2
	feature_2 = 1:
		feature_1 = 0:
			feature_0 = 0:
				prediction: 1
			feature_0 = 1:
				prediction: 1
			feature_0 = 2:
				prediction: 0
		feature_1 = 1:
			prediction: 0
feature_3 = 1:
	prediction: 1
</pre>

In [622]:
#Debug on pizza
pizza_data = pd.DataFrame({
    "Meat" : ['Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'N'],
    "Crust" : ['Thin', 'Deep', 'Stuffed', 'Stuffed', 'Deep', 'Deep', 'Thin', 'Deep', 'Thin'],
    'Veg' : ['N', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N'],
    'Quality' : ['Great', 'Bad', 'Good', 'Great', 'Good', 'Great', 'Good', 'Good', 'Bad']
})

np_data = np.array(pizza_data)

X = np_data[:,0:-1]
y = np_data[:, -1]

dt = DTClassifier(np.array(["Meat", "Crust", "Veg"]))
dt.fit(X, y)

DTClassifier(feature_names=array(['Meat', 'Crust', 'Veg'], dtype='<U5'))

In [623]:
# Load debug training data 
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))

np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))
features = np.array(df.columns)
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

dt = DTClassifier(features)
dt = dt.fit(X,y)

# Load debug test data
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses_test.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))

np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))
y = le.transform(y)

!curl https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv --output dataset.csv
true_results = genfromtxt('dataset.csv', delimiter=',')
# Predict and compute model accuracy
print(dt.score(X, y))
# Print the information gain of every split you make.
print(dt.split_gains)



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2890  100  2890    0     0   7391      0 --:--:-- --:--:-- --:--:--  7391
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2839  100  2839    0     0  22531      0 --:--:-- --:--:-- --:--:-- 22531
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   144  100   144    0     0    503      0 --:--:-- --:--:-- --:--:--   501
0.3333333333333333
[0.54879494 0.77042604 0.31668909 1.         0.45914792 0.91829583]


In [377]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo.arff)

Test your model on this dataset: [Evaluation Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo_test.arff)

Parameters:
(optional) counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)

---
Print out your accuracy on the evaluation test dataset.

Print out the information gain of every split you make.

In [682]:
# Load evaluation data 
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))

np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))
features = np.array(df.columns)
dt = DTClassifier(features)
dt = dt.fit(X,y)

# Load evaluation test data
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo_test.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))

np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))

# Print results
print(dt.score(X, y))
# 0.147
print(dt.split_gains)
# [1.3630469  0.68920199 0.86312057 0.72192809 0.88654089 0.69621226 0.98522814 0.82562653 0.72192809]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6683  100  6683    0     0  28317      0 --:--:-- --:--:-- --:--:-- 28317
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  171k  100  171k    0     0   410k      0 --:--:-- --:--:-- --:--:--  410k
0.147
[1.3630469  0.68920199 0.86312057 0.72192809 0.88654089 0.69621226
 0.98522814 0.82562653 0.72192809]


## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [625]:
# Write a function that implements 10-fold cross validation
def ten_fold_validation(dt, X, y):
    accuracies = np.array([])
    X, y = shuffle_data(X,y)
    split_X = np.array_split(X,10)
    indx = 0
    for array in split_X:
        X_test = X[indx:indx+array.shape[0], :]
        y_test = y[indx:indx+array.shape[0]]
        X_train = np.delete(X, np.s_[indx:indx+array.shape[0]],0)
        y_train = np.expand_dims(np.delete(y, np.s_[indx:indx+array.shape[0]]), 0).T
        dt.fit(X_train, y_train)
        accuracies = np.append(accuracies, dt.score(X_test, y_test))
        indx += array.shape[0]
    return accuracies

##  2.2 Cars Dataset
- Use this [Cars Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/cars.arff)

In [627]:
# Use 10-fold CV on Cars Dataset
# Load cars dataset
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/cars.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))

np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))
features = np.array(df.columns)
dt = DTClassifier(features)
accuracies = ten_fold_validation(dt, X, y)
print("individual accuracies", accuracies)
print("average accuracy", np.average(accuracies))
# Accuracies from one run

# accuracies [0.89595376 0.89595376 0.9017341  0.9132948  0.90751445 0.92485549
# 0.84393064 0.87861272 0.93023256 0.87209302]

#average accuracy ~89.6%

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55393  100 55393    0     0   365k      0 --:--:-- --:--:-- --:--:--  363k
individual accuracies [0.91907514 0.89595376 0.9132948  0.87861272 0.86127168 0.89017341
 0.9017341  0.85549133 0.9127907  0.87790698]
average accuracy 0.890630461083479


## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)
- Note that you will need to support unknown attributes in the voting data set. 

In [679]:
# Used 10-fold CV on Voting Dataset
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))

np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))
features = np.array(df.columns)
dt = DTClassifier(features)
accuracies = ten_fold_validation(dt, X, y)
print("individual accuracies", accuracies)
print("average accuracy", np.average(accuracies))
# Report Training and Test Classification Accuracies

# individual accuracies [0.95454545 0.93181818 0.86363636 0.93181818 0.97727273 0.97674419
# 0.95348837 0.90697674 0.93023256 0.97674419]

# Report Average Test Accuracy
#average accuracy ~94.0%

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40261  100 40261    0     0   295k      0 --:--:-- --:--:-- --:--:--  293k
individual accuracies [0.93181818 0.93181818 0.93181818 0.95454545 0.93181818 0.93023256
 0.93023256 0.93023256 0.97674419 0.93023256]
average accuracy 0.9379492600422832


## 2.4 Discuss Your Results

- Summarize your results from both datasets, and discuss what you observed. 
- A fully expanded tree will often get 100% accuracy on the training set. Why does this happen and in what cases might it not?  

I was able to get close to 90% accuracy on the cars dataset and 93-94% accuracy on the voting dataset.
I have added my individual and average accuracies from doing the ten fold validation in the cells above.

A fully expanded tree can get to 100% accuracy on the training set because it can overfit. If the training set contains pure nodes when you split on all of the features because of low data, you can fit to that and get that accuracy on the training set but it might not be performant on real-world data.

It might not get to 100% if you have rows with very similar data with different targets. Then you end up choosing the most common one, which will not be accurate for the remainding data.

## 3. (15%) For each of the two problems above, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

## 3.1 Discuss what the decision tree induced on the cars dataset has learned

The tree uses safety as its starting point. If the safety is low, we already know that the car in unacceptable. 
At the next layer down after splitting on med and high safety, if the car only has the capacity for two persons, it is also unacceptable. The next biggest attributes to split on are capacity, buying cost, and maintenance cost.

As you go down the tree it gets more complicated, but here are a few gneral observations:
    If a 4 person car's cost is low and maintenance is not very high, it is probably at least acceptable.
    A high buying cost is fine if the maintenance is lower than high.

## 3.2 Discuss what the decision tree induced on the voting dataset has learned

The most important decision in the voting preference on physician-fee-freeze. The unknown value is helpful here because an unknown vote can quickly predict the political party based on the mx-missle and anti-satellite-test-ban
votes.

Going down the other paths takes a few vote splits to start predicting the output class.
If no on the physician-fee-freeze,it can predict on y or ? of the adoption-of-the-budget-resolution vote, otherwise it goes down to education spending and so on.

With a yes on physician-fee-freeze, the tree splits on synfuels-corporation-cutback, then export-administration-act-south-africa and adoption-of-the-budget-resolution.It can predict republican on a n or y on the export vote, otherwise it needs to go down deeper.

## 3.3 How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

I treated unknown attritutes as their own value type because the unknown classification in this dataset 
did not mean that the voting outcome was necessarily unknown. It signified voted present, voted
present to avoid conflict of interest, and did not vote or unknown disposition. This might actually 
be valuable when predicting the target, so I kept the values.

## 4.1 (10%) Use SciKit Learn's decision tree on the voting dataset and compare your results. Try different parameters and report what parameters perform the best on the test set. 

### 4.1.1 SK Learn on Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)

In [678]:
# Use SK Learn's Decision Tree to learn the voting dataset
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff --output dataset.arff
data = arff.loadarff('dataset.arff')
df = pd.DataFrame(data[0])
df = df.apply(lambda x: x.str.decode('utf-8'))
np_data = np.array(df)
X = np_data[:,0:-1]
y = np.reshape(np_data[:,-1], (-1, 1))
features = np.array(df.columns)
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

indx = 0
for col in X.T:
    le.fit(col)
    X[:, indx] = le.transform(col)
    indx += 1

clf = DecisionTreeClassifier(criterion='gini', min_samples_split=16)
clf.fit(X,y)
#ten_fold_validation(clf, X, y)
accuracies = cross_val_score(clf, X, y.ravel(), cv=10)
print("individual accuracies", accuracies)
print("average accuracy", np.average(accuracies))
# Explore different parameters

# Report results

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40261  100 40261    0     0   126k      0 --:--:-- --:--:-- --:--:--  126k
individual accuracies [0.97727273 0.97727273 0.97727273 0.93181818 1.         0.93023256
 1.         0.90697674 0.90697674 0.95348837]
average accuracy 0.9561310782241014


/home/brandon/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


With the default DecisionTreeClassifier I got around 94% accuracy. I was able to bump it up to 95.6% by
increasing min samples split to 16. The other parameters I tried such as criterion, splitter, max_depth, and min samples leaf didn't change much or decreased the accuracy.

My own decision tree was slightly less accurate, usually averaging around 93-94% accuracy, but if I tuned my own to work with these parameters I would probably get about the same accuracy. 

## 4.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the SK decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

In [680]:
# Use SciKit Learn's Decision Tree on a new dataset
notebook_path = os.path.abspath("lab_3_decision_tree.ipynb")
train_csv = os.path.join(os.path.dirname(notebook_path), "Employee-Attrition.csv")
df=pd.read_csv(train_csv, sep=',',header=None)
np_data = np.array(df)

X = np_data[1:,0:-1]
y = np.reshape(np_data[1:,-1], (-1, 1))

indx = 0
for col in X.T:
    le.fit(col)
    X[:, indx] = le.transform(col)
    indx += 1
clf = DecisionTreeClassifier(criterion="entropy", min_samples_split=100, min_samples_leaf=20)
#clf = DecisionTreeClassifier()
clf.fit(X,y)
accuracies = cross_val_score(clf, X, y.ravel(), cv=10)
print("average accuracy", np.average(accuracies))

# I used the Predicting Employee Attrition dataset found on Kaggle

# I was able to increase the average accuracy of the Decision tree from 76.59% to 85.85% by changing the
# criterion to entropy, the min samples to split on to 100 and the min samples for a leaf to 20.
# The other values I experimented with decreased the accuracy.

average accuracy 0.8585034013605443


## 6. (5%) Visualize your decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first several levels (e.g. top 5).

In [681]:
# Include decision tree visualization here
#with open("clf.dot", "w") as f:
#    f = tree.export_graphviz(clf, out_file=f, max_depth=3)

#                              Overtime
#                       True           False
#                Total Working Years <=1.5    Job Level <= .5
#            True         False                        True                  False
#        No Attrition     StockOptionLevel <= .5    Monthly Income <=462        JobRole <6.5
#                                                   No Attrition   Attrition

    
# Discuss what the model has learned

# The model has learned that the best indicators for predicting employee attrition. The best indicator is
# whether or not someone does overtime. Total working years, Job level, job role, stock option level, 
# and monthly income are also important.
# An easy prediction is that if someone is working overtime and has worked at the firm for less than two years,
# they are likely to stay with the company. 


# I only included the top of the tree because it was so large, and I made
# a pdf with export graph viz but I don't think I can imbed it.

## 7. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).